# Airline AI Assistant

In [1]:
# imports
import os
import json
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
import gradio as gr

In [2]:
# Initializing

load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
model = "gemini-2.0-flash"
gemini = ChatGoogleGenerativeAI(
    model = model,
    temperature  = 0.5
)

In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI."
system_message += "Give short, courteous answer, no more than 1 sentence."
system_message += "Always be accurate, if you   dont know the answer, say so."

In [7]:
def chat(message, history):
    messages = [
        SystemMessage(content=system_message)
    ]

    for turn in history:
        if turn["role"] == "user":
            messages.append(HumanMessage(content=turn["content"]))
        elif turn["role"] == "assistant":
            messages.append(AIMessage(content=turn["content"]))
            
    messages.append(HumanMessage(content=message))

    # Streaming
    response = ""

    for chunk in gemini.stream(messages):
        response += chunk.content
        yield response

In [8]:
chat_interface = gr.ChatInterface(
    fn = chat,
    type = "messages",
    title = "Aireline AI Assistant"
)
chat_interface.launch(share = True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/09/28 14:08:18 [W] [service.go:132] login to server failed: i/o deadline reached


## Tools

In [11]:
# Let's start by making a function

ticket_prices = {"london" : "$799", "paris" : "$899", "tokyo" : "$1400", "berlin" : "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for: {destination_city}")
    city = destination_city.lower()

    return ticket_prices.get(city, "Unknown")

In [12]:
get_ticket_price("Berlin")

Tool get_ticket_price called for: Berlin


'$499'

In [19]:
# Dictionary structure to describe our function
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket of this city?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [20]:
# This is included in list of tools:

tools = [{"type" : "function", "function" : price_function}]

### Gemini Response 
```json
{
    "content": "Hello! I can help you learn about flight ticket.",
    "role": "assistant",
    "metadata": {
        "source": "example",
        "timestamp": "2025-09-28T12:00:00Z"
    }
}


In [22]:
def chat(message, history):
    
    messages = [
        SystemMessage(content = system_message)
    ]

    for turn in history:
        if turn['role'] == 'user':
            messages.append(HumanMessage(content = turn['content']))
        if turn['role'] == 'assistant':
            messages.append(AIMessage(content = turn['content']))

    messages.append(HumanMessage(content = message))
    response = gemini.invoke(messages = messages,tools = tools)


    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = gemini.invoke(messages)

        return response.content